In [1]:
from matplotlib.pyplot import draw, fill
import pyrender
import glob
import trimesh
from trimesh import remesh
import pandas as pd
import numpy as np
from trimesh import Trimesh
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import cm
import math
import os
import matplotlib.mlab as mlab
import seaborn as sns
import skeletor as sk
from view_data import view_mesh
import copy
from sklearn.preprocessing import MinMaxScaler
from project_statistics import get_outliers
from create_dataset import load_dataset,dir_to_sorted_file_list
from trimesh.repair import fill_holes
from trimesh.points import PointCloud
import re
import random
import pickle
from scipy.stats import wasserstein_distance
from utils import*

from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
with open('query_database.pkl', 'rb') as f:
    query_database = pickle.load(f)

In [4]:
DB_DIRECTORY = r"full_normalized_benchmark\**\*.off"
mesh_files = list(glob.glob(DB_DIRECTORY,recursive=True))
mesh_files.sort(key=natural_keys)
mean_std = np.load('mean_std.npy')

In [5]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(np.array(query_database.iloc[:,1:]), np.array(query_database['class']))

d = np.zeros((19,19))
class_acc = {}
K = 5
TP = {}
FP = {}
FN = {}
for cls in range(0,19):
    class_acc[cls] = 0
    TP[cls] = 0
    FP[cls] = 0
    FN[cls] = 0
    query_classes = np.array(query_database[query_database['class']==cls].iloc[:,1:])
    num_rows = 0
    for row in query_classes:
        num_rows +=1
        knn_query = np.array(row)[np.newaxis,...]
        knn_neighbors = knn.kneighbors(knn_query, K)[1][0]
        pred_list = (map(lambda x: query_database.loc[x]["class"], knn_neighbors))
        pred_list = list(map(int, pred_list))
        target_list = np.repeat(cls, K)
        d[cls] += [pred_list.count(j) for j in range(0,19)]

    class_acc[cls] = d[cls][cls]/np.sum(d[cls])
    TP[cls] = d[cls][cls]
    FP[cls] = np.sum(d[cls]) - TP[cls]

for i in range(0,19):
    FN[i] = np.sum(d[:,i]) - TP[i]

P = {}
R = {}
F1 = {} 
for i in range(0,19):
    P[i] = 0
    R[i] = 0
    P[i] = TP[i]/(TP[i] + FP[i])
    R[i] = TP[i]/(TP[i] + FN[i])
    F1[i] = 2*((P[i]*R[i])/(P[i]+R[i]))

In [7]:
class_acc

{0: 0.428,
 1: 0.598,
 2: 0.49,
 3: 0.518,
 4: 0.36,
 5: 0.358,
 6: 0.478,
 7: 0.34545454545454546,
 8: 0.474,
 9: 0.416,
 10: 0.658,
 11: 0.478,
 12: 0.53,
 13: 0.398,
 14: 0.402,
 15: 0.64,
 16: 0.356,
 17: 0.348,
 18: 0.22666666666666666}

In [8]:
global_acc = np.mean(list(class_acc.values()))
global_acc

0.44748006379585326

In [9]:
P

{0: 0.428,
 1: 0.598,
 2: 0.49,
 3: 0.518,
 4: 0.36,
 5: 0.358,
 6: 0.478,
 7: 0.34545454545454546,
 8: 0.474,
 9: 0.416,
 10: 0.658,
 11: 0.478,
 12: 0.53,
 13: 0.398,
 14: 0.402,
 15: 0.64,
 16: 0.356,
 17: 0.348,
 18: 0.22666666666666666}

In [10]:
global_p = np.mean(list(P.values()))
global_p

0.44748006379585326

In [11]:
R

{0: 0.45052631578947366,
 1: 0.5155172413793103,
 2: 0.4390681003584229,
 3: 0.4288079470198676,
 4: 0.4166666666666667,
 5: 0.42822966507177035,
 6: 0.5085106382978724,
 7: 0.4453125,
 8: 0.49894736842105264,
 9: 0.4406779661016949,
 10: 0.6266666666666667,
 11: 0.3983333333333333,
 12: 0.47237076648841353,
 13: 0.47607655502392343,
 14: 0.3544973544973545,
 15: 0.5333333333333333,
 16: 0.3938053097345133,
 17: 0.42962962962962964,
 18: 0.22972972972972974}

In [12]:
global_r = np.mean(list(R.values()))
global_r

0.446668794081212

In [13]:
F1

{0: 0.438974358974359,
 1: 0.5537037037037038,
 2: 0.46313799621928164,
 3: 0.46920289855072467,
 4: 0.3862660944206009,
 5: 0.38997821350762524,
 6: 0.49278350515463915,
 7: 0.3890784982935154,
 8: 0.48615384615384616,
 9: 0.42798353909465014,
 10: 0.641951219512195,
 11: 0.4345454545454545,
 12: 0.49952874646559847,
 13: 0.4335511982570806,
 14: 0.37675726335520143,
 15: 0.5818181818181818,
 16: 0.37394957983193283,
 17: 0.3845303867403314,
 18: 0.22818791946308725}

In [14]:
global_f1 = np.mean(list(F1.values()))
global_f1

0.44484645284536894

In [15]:
TP

{0: 214.0,
 1: 299.0,
 2: 245.0,
 3: 259.0,
 4: 180.0,
 5: 179.0,
 6: 239.0,
 7: 171.0,
 8: 237.0,
 9: 208.0,
 10: 329.0,
 11: 239.0,
 12: 265.0,
 13: 199.0,
 14: 201.0,
 15: 320.0,
 16: 178.0,
 17: 174.0,
 18: 17.0}

In [16]:
FP

{0: 286.0,
 1: 201.0,
 2: 255.0,
 3: 241.0,
 4: 320.0,
 5: 321.0,
 6: 261.0,
 7: 324.0,
 8: 263.0,
 9: 292.0,
 10: 171.0,
 11: 261.0,
 12: 235.0,
 13: 301.0,
 14: 299.0,
 15: 180.0,
 16: 322.0,
 17: 326.0,
 18: 58.0}

In [17]:
FN

{0: 261.0,
 1: 281.0,
 2: 313.0,
 3: 345.0,
 4: 252.0,
 5: 239.0,
 6: 231.0,
 7: 213.0,
 8: 238.0,
 9: 264.0,
 10: 196.0,
 11: 361.0,
 12: 296.0,
 13: 219.0,
 14: 366.0,
 15: 280.0,
 16: 274.0,
 17: 231.0,
 18: 57.0}